![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import joblib


## 0. Preprocesamiento de Datos

Los datos de entrenamiento se dividen en datos de entrenamiento y validación. Si decidieron preprocesar los datos (estandarizar, normalizar, imputar valores, etc), estos son correctamente preprocesados al ajustar sobre los datos de entrenamiento (.fit_transform()) y al transformar los datos del set de validación (.transform()). (10 puntos)

In [4]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
print(dataTraining.head
())

   Price  Year  Mileage State       Make      Model
0  34995  2017     9913    FL       Jeep   Wrangler
1  37895  2015    20578    OH  Chevrolet   Tahoe4WD
2  18430  2012    83716    TX        BMW      X5AWD
3  24681  2014    28729    OH   Cadillac  SRXLuxury
4  26998  2013    64032    CO       Jeep   Wrangler


In [5]:
# Visualización datos de test
print(dataTesting.head())

    Year  Mileage State    Make        Model
ID                                          
0   2014    31909    MD  Nissan    MuranoAWD
1   2017     5362    FL    Jeep     Wrangler
2   2014    50300    OH    Ford  FlexLimited
3   2004   132160    WA     BMW            5
4   2015    25226    MA    Jeep        Grand


In [5]:
dataTraining['State'] = dataTraining['State'].str.strip()
# Convertir variables categóricas en variables dummy
dataTraining = pd.get_dummies(dataTraining, columns=['State', 'Make', 'Model'])

# Visualizar los primeros registros del conjunto de datos preprocesado
print(dataTraining.head())

   Price  Year  Mileage  State_AK  State_AL  State_AR  State_AZ  State_CA  \
0  34995  2017     9913     False     False     False     False     False   
1  37895  2015    20578     False     False     False     False     False   
2  18430  2012    83716     False     False     False     False     False   
3  24681  2014    28729     False     False     False     False     False   
4  26998  2013    64032     False     False     False     False     False   

   State_CO  State_CT  ...  Model_Yaris4dr  Model_YarisBase  Model_YarisLE  \
0     False     False  ...           False            False          False   
1     False     False  ...           False            False          False   
2     False     False  ...           False            False          False   
3     False     False  ...           False            False          False   
4      True     False  ...           False            False          False   

   Model_Yukon  Model_Yukon2WD  Model_Yukon4WD  Model_Yukon4dr  Mode

In [6]:
# Definir las características (X) y la variable objetivo (y)
X = dataTraining.drop('Price', axis=1)
y = dataTraining['Price']

# Dividir los datos en conjuntos de entrenamiento y prueba
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Inicializar el modelo con los hiperparámetros ajustados
xgb_model = XGBRegressor(
    booster="gbtree",
    colsample_bytree=1,
    eta=0.3,
    gamma=1,
    max_depth=9,
    max_leaves=0,
    n_estimators=540,
    objective="reg:linear",
    reg_alpha=2.291666666666667,
    reg_lambda=1.7708333333333335,
    tree_method="auto",
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
xgb_model.fit(XTrain, yTrain)

# Imprimir mensaje de finalización
print("Entrenamiento completado.")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:14:13] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


Entrenamiento completado.


In [8]:
columnas = X.columns

In [9]:
columnas

Index(['Year', 'Mileage', 'State_AK', 'State_AL', 'State_AR', 'State_AZ',
       'State_CA', 'State_CO', 'State_CT', 'State_DC',
       ...
       'Model_Yaris4dr', 'Model_YarisBase', 'Model_YarisLE', 'Model_Yukon',
       'Model_Yukon2WD', 'Model_Yukon4WD', 'Model_Yukon4dr', 'Model_tC2dr',
       'Model_xB5dr', 'Model_xD5dr'],
      dtype='object', length=616)

In [11]:
joblib.dump({'modelo': xgb_model, 'columnas': columnas}, 'prediction_car_reg.pkl', compress=3)

['prediction_car_reg.pkl']

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Realizar predicciones en el conjunto de prueba
y_pred = xgb_model.predict(XTest)

# Calcular métricas de rendimiento
mse = mean_squared_error(yTest, y_pred)
mae = mean_absolute_error(yTest, y_pred)
r2 = r2_score(yTest, y_pred)
rmse = np.sqrt(mse)

# Imprimir las métricas de rendimiento
print("Métricas de rendimiento del modelo:")
print("Error cuadrático medio (MSE):", mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)
print("Error absoluto medio (MAE):", mae)
print("Coeficiente de determinación (R^2):", r2)


Métricas de rendimiento del modelo:
Error cuadrático medio (MSE): 11997074.91404786
Raíz del error cuadrático medio (RMSE): 3463.679389615595
Error absoluto medio (MAE): 2171.490903515625
Coeficiente de determinación (R^2): 0.8963045725274584


In [ ]:
# Convertir variables categóricas en variables numéricas
dataTesting_encoded = pd.get_dummies(dataTesting, columns=['State', 'Make', 'Model'])

# Asegurarse de que las columnas del conjunto de prueba coincidan con las del conjunto de entrenamiento
missing_cols = set(XTrain.columns) - set(dataTesting_encoded.columns)
for col in missing_cols:
    dataTesting_encoded[col] = 0
dataTesting_encoded = dataTesting_encoded[XTrain.columns]

# Realizar predicciones en el conjunto de prueba preprocesado
predictions = xgb_model.predict(dataTesting_encoded)
print(dataTesting_encoded)

       Year  Mileage  State_ AK  State_ AL  State_ AR  State_ AZ  State_ CA  \
ID                                                                            
0      2014    31909      False      False      False      False      False   
1      2017     5362      False      False      False      False      False   
2      2014    50300      False      False      False      False      False   
3      2004   132160      False      False      False      False      False   
4      2015    25226      False      False      False      False      False   
...     ...      ...        ...        ...        ...        ...        ...   
99995  2015    82719      False      False      False      False      False   
99996  2015    19711      False      False      False      False      False   
99997  2016    48049      False      False      False      False       True   
99998  2017    15306      False      False      False      False      False   
99999  2013    40393      False      False      Fals

In [ ]:
# Crear un DataFrame con las predicciones para subir a la plataforma
predicciones_XGB = pd.DataFrame({'ID': dataTesting_encoded.index, 'Price': predictions})
predicciones_XGB.to_csv('predicciones_XGB_7.csv', index=False)
print(predicciones_XGB)

          ID         Price
0          0  21179.744141
1          1  37660.210938
2          2  23639.357422
3          3   7963.722168
4          4  31302.402344
...      ...           ...
99995  99995  19815.648438
99996  99996  38392.316406
99997  99997  23235.943359
99998  99998  17244.031250
99999  99999  18472.423828

[100000 rows x 2 columns]


#-------------- FINAL

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Definir rangos para cada hiperparámetro
param_grid = {
    'booster': ['gbtree'],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'eta': [0.1, 0.2, 0.3],
    'gamma': [0, 1, 5],
    'max_depth': [4, 6, 8],
    'max_leaves': [0],  # Considera ajustar este rango si es necesario
    'n_estimators': [50, 100, 200],
    'objective': ['reg:linear'],
    'reg_alpha': [0, 1, 2],
    'reg_lambda': [0, 1, 2],
    'subsample': [0.6, 0.7, 0.8],
    'tree_method': ['auto'],
    'random_state': [42]
}

# Inicializar el modelo
xgb_model = XGBRegressor()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(XTrain, yTrain)

# Obtener el mejor modelo
best_xgb_model = grid_search.best_estimator_

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Imprimir mensaje de finalización
print("Entrenamiento completado.")
